In [1]:
import tensorflow as tf
import cv2
import numpy as np

In [ ]:
tf.io.

In [12]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the
    # unserialized graph_def
    with tf.io.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="")
    return graph

In [20]:
model_graph = load_graph(r"./models/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb")
# outputs = [
#     model_graph.get_tensor_by_name("raw_outputs/class_predictions:0"),
#     model_graph.get_tensor_by_name("raw_outputs/box_encodings:0")
# ]
outputs = [
    model_graph.get_tensor_by_name("detection_boxes:0"),
    model_graph.get_tensor_by_name("detection_scores:0"),
    model_graph.get_tensor_by_name("num_detections:0"),
    model_graph.get_tensor_by_name("detection_classes:0")
]

In [26]:
output_tensors[1].shape

(1, 100)

In [29]:
image = cv2.imread(r"./images/people-counter-image.png").astype(np.float32)
image = cv2.resize(image, (320, 320))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image = (image - 128) / 127.5
image = np.expand_dims(image, 0)

with tf.compat.v1.Session(graph=model_graph) as sess:
    output_tensors = sess.run(outputs, feed_dict={'image_tensor:0': image})

detection_boxes = output_tensors[0]
print(detection_boxes)
detection_scores = output_tensors[1]
print(detection_scores)
num_detections = output_tensors[2]
print(num_detections)
detection_classes = output_tensors[3]
print(detection_classes)

[[[0.19532764 0.49990642 0.7876607  0.6278235 ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.  

In [30]:
output_tensors[0].shape

(1, 100, 4)

In [58]:
output_tensors[1].shape

(1, 100)

In [60]:
output_tensors[3].shape

(1, 100)

In [46]:
(np.argmax(output_tensors[1], axis=1)).nonzero()

(array([], dtype=int64),)

In [51]:
(np.argmax(output_tensors[1], axis=1) == 0).nonzero()

(array([0]),)

In [80]:
(np.ones((1, 1, 100, 7))[0, 0, 0, 0] > 0.5).nonzero()

/Users/jians/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  """Entry point for launching an IPython kernel.


(array([0]),)